# In this notebook we find train type correspondance
Indeed, `route_desc` and `verkehrsmittel_text` aren't the same, so we have to map them together

In [ ]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

#### Imports:

In [ ]:
import networkx as nx
from geopy.distance import distance as geo_distance
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from networkx.algorithms.shortest_paths.weighted import dijkstra_path

#### Load data:

In [ ]:
routes = spark.read.format('orc').load('/data/sbb/timetables/orc/routes')
stop_times = spark.read.format('orc').load('/data/sbb/timetables/orc/stop_times/000000_0')
stops = spark.read.format('orc').load('/data/sbb/timetables/orc/stops/000000_0')
trips = spark.read.format('orc').load('/data/sbb/timetables/orc/trips/000000_0')
actual = spark.read.format('orc').load('/data/sbb/orc/istdaten/')

# Look at possible values

In [ ]:
# Possible `verkehrsmittel_text` values
actual.select(col('verkehrsmittel_text')).distinct().show(100)

In [ ]:
# Possible `route_desc` values
routes.select(col('route_desc')).distinct().show(100)

# Replace value

With the manual checks in the next section, [this pdf](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&ved=2ahUKEwiB_sLqlL3pAhVKXhoKHSUUD28QFjAAegQIARAB&url=https%3A%2F%2Fcompany.sbb.ch%2Fcontent%2Fdam%2Finternet%2Fcorporate%2Ffr%2Fsbb-als-geschaeftspartner%2Fflotte-unterhalt%2Fonestopshop%2FLeistungskatalog-2020.pdf.sbbdownload.pdf&usg=AOvVaw1faXmNvW-PypBiC-Hz7c_W) and online research. We can map `verkehrsmittel_text` to the same values we map `route_desc` to

In [ ]:
translate_route_desc = {
    'TGV': 'TGV',
    'Eurocity': 'EC',
    'tandseilbahn': 'AT',
    'Regionalzug': 'R',
    'RegioExpress': 'RE',
    'S-Bahn': 'S',
    'Luftseilbahn': '',
    'Sesselbahn': '',
    'Taxi': '', # No taxi information
    'Fähre': '',
    'Tram': 'Tram',
    'ICE': 'ICE',
    'Bus': 'Bus',
    'Gondelbahn': '',
    'Nacht-Zug': '', # We don't look at nacht-zug
    'Standseilbahn': 'AT',
    'Auoreisezug': 'ARZ',
    'Eurostar': 'EC',
    'Schiff': '',
    'Schnellzug': 'TGV',
    'Intercity': 'IC',
    'InterRegio': 'IR',
    'Extrazug': 'EXT',
    'Metro': 'Metro'
}

In [ ]:
replace_actual = {
    'BUS': 'Bus', # Buses
    'B': 'Bus',
    'NFB': 'Bus',
    'KB': 'Bus',
    'BAT': 'Bus',
    'Trm': 'Tram', # Trams
    'T': 'Tram',
    'TRAM': 'Tram',
    'ATZ': 'ARZ', #AutoZug
    'D': 'RE', # Regional
    'RB': 'R',
    'M': 'Metro', # Metro
    'ICE': 'IC', # InterCityExpress, but routes.txt doesn't have that category
    'IRE': 'IR', # InterRegioExpress, but routes.txt doesn't have that category
    'BN': '', # Night
    'TN': '',
    'SN': '',
    'BT': '',
    'VAE': '', # Panorama trains in the Alps
    'PE': '',
    'TER': '', # France
    'TE2': '',
    'RJX': '', # International
    'null': '' # Other
}

In [ ]:
@udf("string")
def replace_verkehrsmittel_text(text):
    if text in replace_actual.keys():
        return replace_actual[text]
    else:
        return text

In [ ]:
# Look at the number of data we have for each `verkehrsmittel_text`, as we can see we covered all the big categories
counts.select(replace_verkehrsmittel_text(col('verkehrsmittel_text')).alias('verkehrsmittel_text'), col('count'))\
        .groupBy('verkehrsmittel_text').sum()\
        .sort('sum(count)', ascending=False).show(100)

# Manual checks

## Check T

In [ ]:
actual.where(col('verkehrsmittel_text') == 'T').where(col('haltestellen_name') == 'Zürich, Bachmattstrasse').select(col('fahrt_bezeichner')).limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'T').where(col('fahrt_bezeichner') == '85:3849:113276-32002-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> T is Tram

## Check B

In [ ]:
actual.where(col('verkehrsmittel_text') == 'B').select(col('fahrt_bezeichner')).limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'B').where(col('fahrt_bezeichner') == '85:827:280700-02132-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> B is Bus

## Check NFB

In [ ]:
actual.where(col('verkehrsmittel_text') == 'NFB').select(col('fahrt_bezeichner')).limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'NFB').where(col('fahrt_bezeichner') == '85:870:51001')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> NFB is Bus, NF actually means 'NiederFlurzüge' meaning the bus can be used by disabled

## Check R

In [ ]:
actual.where(col('verkehrsmittel_text') == 'R').select(col('fahrt_bezeichner')).limit(20).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'R').where(col('fahrt_bezeichner') == '83:3270__:28318:000')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .sort('ankunftszeit').show(20, False)

#### From sbb.ch -> R is regional train

## Check M

In [ ]:
actual.where(col('verkehrsmittel_text') == 'M').select(col('fahrt_bezeichner')).limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'M').where(col('fahrt_bezeichner') == '85:151:TL070-4506262507106362')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> M is Metro

## Check BN

In [ ]:
actual.where(col('verkehrsmittel_text') == 'BN').select(col('fahrt_bezeichner')).limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'BN').where(col('fahrt_bezeichner') == '85:773:9232-01353-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> These are night weekend buses, so we don't want to count them. We only look at reasonable hours

## Check TN

In [ ]:
actual.where(col('verkehrsmittel_text') == 'TN').select(col('fahrt_bezeichner')).limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'TN').where(col('fahrt_bezeichner') == '85:37:13059-00607-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> These are night weekend trams, so we don't want to count them. We only look at reasonable hours

## Check KB

In [ ]:
actual.where(col('verkehrsmittel_text') == 'KB').select(col('fahrt_bezeichner')).limit(20).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'KB').where(col('fahrt_bezeichner') == '85:870:51013')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> Another type of bus

## Check SN

In [ ]:
actual.where(col('verkehrsmittel_text') == 'SN').select(col('fahrt_bezeichner')).limit(20).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'SN').where(col('fahrt_bezeichner') == '85:46:907:000')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> Night S, so we don't count them

## Check VAE

In [ ]:
actual.where(col('verkehrsmittel_text') == 'VAE').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'VAE').where(col('fahrt_bezeichner') == '85:82:72564:002')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> VAE is Voralpen-Express so trains in the Alps for tourism. We don't need to count them since we consider stations around Zürich

## Check BAT

In [ ]:
actual.where(col('verkehrsmittel_text') == 'BAT').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'BAT').where(col('fahrt_bezeichner') == '85:183:60:000')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

#### From sbb.ch -> Some type of bus

## Check D

#### From sbb.ch -> D is Direct, like RegioExpress

## Check TER

In [ ]:
actual.where(col('verkehrsmittel_text') == 'TER').select(col('fahrt_bezeichner')).distinct().limit(20).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'TER')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .sort('ankunftszeit').show(20, False)

#### From sbb.ch -> TER is a French railway, so we remove it since we look at Zürich

## Check EXT

In [ ]:
actual.where(col('verkehrsmittel_text') == 'EXT').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'EXT')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '20.11.2019 00:00').sort('ankunftszeit').show(20, False)

#### This looks very much like ExtraZug

## Check RB

#### From sbb.ch -> RB is RegionalBahn

## Check BT

In [ ]:
actual.where(col('verkehrsmittel_text') == 'BT').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'BT').where(col('fahrt_bezeichner') == '85:823:689210-09073-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .sort('ankunftszeit').show(20, False)

#### From sbb.ch -> BT is a night tram, so we ignore it

## Check BAV

In [ ]:
actual.where(col('verkehrsmittel_text') == 'BAV').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

In [ ]:
actual.where(col('verkehrsmittel_text') == 'BAV').where(col('fahrt_bezeichner') == '85:185:18:000')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .sort('ankunftszeit').show(20, False)

#### From sbb.ch -> BT is a night tram, so we ignore it